In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.optim as optim


In [2]:
data_path = '/content/drive/MyDrive/ELEG605/ELEG605/prj/balanced_multiclass_data.csv'

In [3]:
# Function to process chunks
def process_chunks(file_path, chunksize=100000, subset_frac=0.1):
    sampled_chunks = []
    total_rows = 0
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        # Count total rows
        total_rows += len(chunk)
        # Stratified sampling
        stratified_chunk = chunk.groupby('Attack_Class', group_keys=False).apply(
            lambda x: x.sample(frac=subset_frac, random_state=42)
        )
        sampled_chunks.append(stratified_chunk)
    # Combine sampled chunks
    sampled_data = pd.concat(sampled_chunks, ignore_index=True)
    print(f"Processed {total_rows} rows. Sampled down to {len(sampled_data)} rows.")
    return sampled_data

subset_df = process_chunks(data_path, chunksize=100000, subset_frac=0.1)


<ipython-input-3-38d8a3780f75>:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stratified_chunk = chunk.groupby('Attack_Class', group_keys=False).apply(
<ipython-input-3-38d8a3780f75>:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stratified_chunk = chunk.groupby('Attack_Class', group_keys=False).apply(
<ipython-input-3-38d8a3780f75>:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping 

Processed 9472426 rows. Sampled down to 947242 rows.


In [4]:

# Encode labels
label_encoder = LabelEncoder()
subset_df['Attack_Class_Label'] = label_encoder.fit_transform(subset_df['Attack_Class'])

X = subset_df.drop(columns=['Attack_Class', 'Attack_Class_Label'])
y = subset_df['Attack_Class_Label']


In [5]:
print(f"X.shape", X.shape)
print(f"y.shape", y.shape)

X.shape (947242, 116)
y.shape (947242,)


In [6]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

# Define base models
base_models = [
    ('lr', LogisticRegression(penalty='l2', C=1.0, max_iter=100, random_state=42)),
    ('dt', DecisionTreeClassifier(max_depth=10, min_samples_split=10, random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42))
]

In [9]:
# Function to train base models and collect meta-features
def train_base_models(X, y, base_models, num_classes, num_folds=3):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    meta_train_list, meta_test_list, y_train_list, y_test_list = [], [], [], []

    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        print(f"=== Fold {fold + 1} ===")
        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]

        num_base_models = len(base_models)
        meta_train = np.zeros((X_train_fold.shape[0], num_base_models * num_classes))
        meta_test = np.zeros((X_test_fold.shape[0], num_base_models * num_classes))

        for i, (name, model) in enumerate(base_models):
            print(f"Training {name} on fold {fold + 1}...")
            model.fit(X_train_fold, y_train_fold)
            y_pred_proba_train = model.predict_proba(X_train_fold)
            y_pred_proba_test = model.predict_proba(X_test_fold)

            meta_train[:, i*num_classes:(i+1)*num_classes] = y_pred_proba_train
            meta_test[:, i*num_classes:(i+1)*num_classes] = y_pred_proba_test

            # Print classification report for base models
            y_pred_test = model.predict(X_test_fold)
            print(f"Classification Report for {name} on Test Data:")
            print(classification_report(y_test_fold, y_pred_test))

        meta_train_list.append(meta_train)
        meta_test_list.append(meta_test)
        y_train_list.append(y_train_fold)
        y_test_list.append(y_test_fold)

    return np.vstack(meta_train_list), np.vstack(meta_test_list), np.concatenate(y_train_list), np.concatenate(y_test_list)

# Define base models
base_models = [
    ('lr', LogisticRegression(max_iter=200, random_state=42)),
    ('dt', DecisionTreeClassifier(max_depth=10, random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=50, random_state=42))
]

# Train base models and collect meta-features
num_classes = len(np.unique(y))
X_train_meta, X_test_meta, y_train_meta, y_test_meta = train_base_models(X_scaled, y.to_numpy(), base_models, num_classes)


=== Fold 1 ===
Training lr on fold 1...


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classification Report for lr on Test Data:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     27206
           1       0.84      0.70      0.76     25816
           2       0.65      0.81      0.72     18066
           3       1.00      1.00      1.00     25200
           4       0.00      0.00      0.00     36718
           5       0.51      1.00      0.68     38566
           6       0.96      0.97      0.97     33253
           7       1.00      1.00      1.00     25400
           8       1.00      1.00      1.00     26046
           9       0.93      0.90      0.91     30887
          10       0.89      0.92      0.90     28590

    accuracy                           0.83    315748
   macro avg       0.80      0.84      0.81    315748
weighted avg       0.77      0.83      0.79    315748

Training dt on fold 1...
Classification Report for dt on Test Data:
              precision    recall  f1-score   support

           0       1.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classification Report for lr on Test Data:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     27238
           1       0.84      0.71      0.77     25440
           2       0.67      0.81      0.73     18221
           3       1.00      1.00      1.00     25079
           4       0.01      0.00      0.00     37171
           5       0.51      1.00      0.67     38524
           6       0.96      0.98      0.97     32897
           7       0.99      1.00      1.00     25210
           8       1.00      0.99      1.00     26308
           9       0.93      0.88      0.91     30849
          10       0.88      0.92      0.90     28810

    accuracy                           0.82    315747
   macro avg       0.80      0.84      0.81    315747
weighted avg       0.77      0.82      0.79    315747

Training dt on fold 2...
Classification Report for dt on Test Data:
              precision    recall  f1-score   support

           0       1.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classification Report for lr on Test Data:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     27020
           1       0.84      0.70      0.77     25528
           2       0.66      0.81      0.73     18088
           3       1.00      1.00      1.00     25044
           4       0.01      0.00      0.00     36985
           5       0.51      1.00      0.68     38603
           6       0.96      0.98      0.97     33129
           7       0.99      1.00      1.00     25503
           8       1.00      0.99      1.00     26265
           9       0.94      0.89      0.91     30741
          10       0.89      0.92      0.90     28841

    accuracy                           0.83    315747
   macro avg       0.80      0.84      0.81    315747
weighted avg       0.77      0.83      0.79    315747

Training dt on fold 3...
Classification Report for dt on Test Data:
              precision    recall  f1-score   support

           0       1.

In [19]:

import torch.nn.functional as F

class MLPMetaLearner(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLPMetaLearner, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)  # Increase neurons for first layer
        self.bn1 = nn.BatchNorm1d(256)        # Add batch normalization
        self.dropout1 = nn.Dropout(0.3)       # Add dropout

        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.3)

        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.dropout3 = nn.Dropout(0.3)

        self.fc4 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.dropout1(F.leaky_relu(self.bn1(self.fc1(x)), negative_slope=0.01))
        x = self.dropout2(F.leaky_relu(self.bn2(self.fc2(x)), negative_slope=0.01))
        x = self.dropout3(F.leaky_relu(self.bn3(self.fc3(x)), negative_slope=0.01))
        return F.softmax(self.fc4(x), dim=1)

In [20]:

# Train and evaluate the meta-learner
def train_meta_learner(X_train_meta, X_test_meta, y_train_meta, y_test_meta, num_classes, device):
    # Define and train the meta-learner
    input_size = X_train_meta.shape[1]
    meta_learner = MLPMetaLearner(input_size=input_size, num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(meta_learner.parameters(), lr=0.001)

    print("Training meta-learner...")
    for epoch in range(200):  # Train for 500 epochs
        optimizer.zero_grad()
        outputs = meta_learner(X_train_meta)
        loss = criterion(outputs, y_train_meta)
        loss.backward()
        optimizer.step()
        print(f"Epoch [{epoch+1}/200], Loss: {loss.item():.4f}")

    # Evaluate meta-learner
    print("Evaluating meta-learner...")
    meta_learner.eval()
    with torch.no_grad():
        y_pred_test = meta_learner(X_test_meta)
        y_pred_test_class = torch.argmax(y_pred_test, dim=1).cpu().numpy()

    print("Classification Report for Meta-Learner:")
    print(classification_report(y_test_meta.cpu().numpy(), y_pred_test_class))

# Convert meta-features to tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train_meta = torch.tensor(X_train_meta, dtype=torch.float32).to(device)
X_test_meta = torch.tensor(X_test_meta, dtype=torch.float32).to(device)
y_train_meta = torch.tensor(y_train_meta, dtype=torch.long).to(device)
y_test_meta = torch.tensor(y_test_meta, dtype=torch.long).to(device)

# Train and evaluate meta-learner
train_meta_learner(X_train_meta, X_test_meta, y_train_meta, y_test_meta, num_classes, device)


<ipython-input-20-43e297da2bdc>:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_meta = torch.tensor(X_train_meta, dtype=torch.float32).to(device)
<ipython-input-20-43e297da2bdc>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_meta = torch.tensor(X_test_meta, dtype=torch.float32).to(device)
<ipython-input-20-43e297da2bdc>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_meta = torch.tensor(y_train_meta, dtype=torch.long).to(device)
<ipython-input-20-43e297da2bdc>:33: UserWarning: To copy construct from a tensor,

Training meta-learner...
Epoch [1/200], Loss: 2.3951
Epoch [2/200], Loss: 2.3584
Epoch [3/200], Loss: 2.3174
Epoch [4/200], Loss: 2.2768
Epoch [5/200], Loss: 2.2400
Epoch [6/200], Loss: 2.2081
Epoch [7/200], Loss: 2.1809
Epoch [8/200], Loss: 2.1571
Epoch [9/200], Loss: 2.1359
Epoch [10/200], Loss: 2.1168
Epoch [11/200], Loss: 2.0988
Epoch [12/200], Loss: 2.0824
Epoch [13/200], Loss: 2.0673
Epoch [14/200], Loss: 2.0531
Epoch [15/200], Loss: 2.0398
Epoch [16/200], Loss: 2.0278
Epoch [17/200], Loss: 2.0166
Epoch [18/200], Loss: 2.0062
Epoch [19/200], Loss: 1.9965
Epoch [20/200], Loss: 1.9874
Epoch [21/200], Loss: 1.9789
Epoch [22/200], Loss: 1.9708
Epoch [23/200], Loss: 1.9628
Epoch [24/200], Loss: 1.9552
Epoch [25/200], Loss: 1.9479
Epoch [26/200], Loss: 1.9406
Epoch [27/200], Loss: 1.9334
Epoch [28/200], Loss: 1.9262
Epoch [29/200], Loss: 1.9194
Epoch [30/200], Loss: 1.9123
Epoch [31/200], Loss: 1.9054
Epoch [32/200], Loss: 1.8986
Epoch [33/200], Loss: 1.8914
Epoch [34/200], Loss: 1.884